<a href="https://colab.research.google.com/github/fmipython/PythonCourse2022/blob/17_c_code_in_python/17%20-%20Using%20C%20code%20in%20Python/17%20-%20Using%20C%20code%20in%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Използване на C код в Python

План на лекцията:
- [ ] Защо C код в Python ?
- [ ] Как работят C библиотеките ?
- [ ] Простичка функция
- [ ] Структури
- [ ] Скорост
- [ ] Примери
- [ ] Задачи

## Защо C код в Python ?

## Как работят C библиотеките ?

## Простичка функция

## Структури

## Скорост

## Примери

## Задачи